In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
accidents = pd.read_csv('./US_Accidents_March23.csv')
#The code below is done only because my laptop cannot handle the sheer volume of data. 
#In a normal situation this would not occur.
accidents = accidents.head(2000)

In [ ]:
accidents.head()

In [ ]:
accidents.describe()

In [ ]:
%load_ext sql
connection = sqlite3.connect('helcim.db')
accidents.to_sql(name = 'accidents', con = connection, if_exists =
'replace')
# to check if connection works, run
pd.read_sql_query('select count(*) from accidents', connection)


In [ ]:
#Extract data into df (Q2)
df =pd.read_sql_query("Select * from accidents",connection)
df.describe()

In [ ]:
df.info()
#Q2
# Initial Insight- Data set has many missing values. It also has some data integrity problem (explained in the pdf documentation).
# There is a datetime component to the dataset, which makes me think I would need to split the data respecting the dates.
# The dataset contains some data from the end of February 2023.
# It has many samples which is ideal for a statistical model. But it also contains many values within each variable (eg Weather_Condition is categorical but not a fixed value)


In [ ]:
#Explanation of each of the query can be found within the pdf documentation.
# Rename of features explained in documentation
database_query ="""
DELETE FROM accidents WHERE Timezone ='' or Timezone IS NULL or Timezone like 'K%' or Timezone like 'k%' or Timezone = 'None';
UPDATE accidents SET Timezone = 'US/Central' WHERE Timezone = 'Central' or Timezone ='us/central' or Timezone='central';
UPDATE accidents SET Timezone = 'US/Eastern' WHERE Timezone = 'Eastern' or Timezone ='us/eastern'or Timezone='eastern';
UPDATE accidents SET Timezone = 'US/Pacific' WHERE Timezone = 'Pacific' or Timezone ='us/pacific'or Timezone='pacific';
UPDATE accidents SET Timezone = 'US/Mountain' WHERE Timezone = 'Mountain' or Timezone ='us/mountain'or Timezone='mountain';
ALTER TABLE accidents ADD COLUMN Timezone_UTC REAL;
UPDATE accidents SET Timezone_UTC = -5 WHERE Timezone ='US/Eastern';
UPDATE accidents SET Timezone_UTC = -6 WHERE Timezone ='US/Central';
UPDATE accidents SET Timezone_UTC = -7 WHERE Timezone ='US/Mountain';
UPDATE accidents SET Timezone_UTC = -8 WHERE Timezone ='US/Pacific';
ALTER TABLE accidents ADD COLUMN Min_Temp REAL;
ALTER TABLE accidents ADD COLUMN Max_Temp REAL;
ALTER TABLE accidents ADD COLUMN Median_Temp REAL;
ALTER TABLE accidents ADD COLUMN Avg_Temp REAL;
ALTER TABLE accidents ADD COLUMN Temp_Difference REAL;
ALTER TABLE accidents RENAME COLUMN 'Temperature_Range(F)' TO Temperature_Range_F;
ALTER TABLE accidents RENAME COLUMN 'Humidity(%)' TO Humidity_Percentage;
ALTER TABLE accidents RENAME COLUMN 'Precipitation(in)' to Precipitation_in;
ALTER TABLE accidents RENAME COLUMN 'Wind_Chill(F)' to Wind_Chill_F;
ALTER TABLE accidents RENAME COLUMN 'Pressure(in)' to Pressure_in;
ALTER TABLE accidents RENAME COLUMN 'Visibility(mi)' to Visibility_mi;
ALTER TABLE accidents RENAME COLUMN 'Wind_Speed(mph)' to Wind_Speed_mph;
ALTER TABLE accidents RENAME COLUMN 'Distance(mi)' to Distance_mi;
UPDATE accidents SET Min_Temp = SUBSTR(TRIM (Temperature_Range_F),1,INSTR(TRIM(Temperature_Range_F),'-')-1);
UPDATE accidents SET Max_Temp = SUBSTR(TRIM (Temperature_Range_F),INSTR(TRIM(Temperature_Range_F),'-')+1);
UPDATE accidents SET Median_Temp =(Min_Temp+Max_Temp)/2;
Update accidents as a1 Set Median_Temp =( Select AVG(a2.Median_Temp) from accidents as a2  where a1.County = a2.County and a1.Weather_Condition =a2.Weather_Condition) where Median_Temp ISNULL;
Update accidents as a1 Set Median_Temp =( Select AVG(a2.Median_Temp) from accidents as a2  where a1.County = a2.County) where Median_Temp ISNULL;
Update accidents as a1 Set Median_Temp =( Select AVG(a2.Median_Temp) from accidents as a2  where a1.State = a2.State) where Median_Temp ISNULL;
Update accidents Set Median_Temp = (Median_Temp-32)*5/9;
ALTER TABLE accidents ADD COLUMN Start_Hour Int;
Update accidents Set Start_Hour = SUBSTR(TRIM (Start_Time),1,INSTR(TRIM(Start_Time),':')-1);
DELETE FROM accidents WHERE Start_Hour>24;
Update accidents as a1 Set Avg_Temp =( Select AVG(a2.Median_Temp) from accidents as a2  where a1.City = a2.City and a1.Start_Hour=a2.Start_Hour);
Update accidents Set Temp_Difference = Median_Temp - Avg_Temp;
ALTER TABLE accidents ADD COLUMN Road_Utility Int;
Update accidents Set Road_Utility = Amenity+Bump+Crossing+Give_Way+Junction+No_Exit+Railway+Roundabout+Station+Stop+Traffic_Calming+Traffic_Signal+Turning_Loop;



"""

In [ ]:
connection.executescript(database_query)

In [ ]:
connection.commit()

In [ ]:
df =pd.read_sql_query("Select * from accidents",connection)

In [ ]:
connection.close()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
#Convert into datetime format - till seconds (although seconds do not exist in the df)
df['Weather_Timestamp']=df['Weather_Timestamp'].astype('datetime64[s]')

In [ ]:
#Split data into dates and times
df['Dates'] = pd.to_datetime(df['Weather_Timestamp']).dt.date
df['Time'] = pd.to_datetime(df['Weather_Timestamp']).dt.time

In [ ]:
#Get count of accidents per county per time
df['Count_County_Date'] = df.groupby(['County', 'Dates'])['County'].transform('count')

In [ ]:
#Get count of accidents per state per time
df['Count_State_Date'] = df.groupby(['State', 'Dates'])['State'].transform('count')

In [ ]:
fig,ax=plt.subplots(figsize=(20,15))
sns.lineplot(x = "Dates", y = "Median_Temp", data = df,hue='State') 

In [ ]:
fig,ax=plt.subplots(figsize=(20,15))
sns.lineplot(x = "Dates", y = "Count_County_Date", data = df,hue='State') 

In [ ]:
fig,ax=plt.subplots(figsize=(20,15))
sns.lineplot(x = "Dates", y = "Count_State_Date", data = df,hue='State') 

In [ ]:
fig,ax=plt.subplots(figsize=(20,15))
sns.lineplot(x = "Dates", y = "Humidity_Percentage", data = df,hue='State') 

In [ ]:
fig,ax=plt.subplots(figsize=(20,15))
sns.lineplot(x = "Dates", y = "Precipitation_in", data = df,hue='State') 

In [ ]:
fig,ax=plt.subplots(figsize=(20,15))
sns.lineplot(x = "Start_Lat", y = "Severity", data = df) 

In [ ]:
#Uisng regression as the labels are continuous. 

#Get rid of useless data

#Dropping End time. I suspect that the end time >24 could be next day. 

df =df.drop(['index','Row_Num','ID','Source','Description','End_Time','End_Lng','Time','County','City','Zipcode','Street','Airport_Code','End_Lat'],axis=1)

In [ ]:

import sklearn.metrics
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

In [ ]:
#Need to sort the timestamp so that it is a time series split.
df =df.sort_values(by="Dates")

In [ ]:
#Check for missing values
round((df.isnull().sum() * 100/ len(df)),2).sort_values(ascending=False)

In [ ]:
#Split data
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [ ]:
#Convert dataset into training and testing (validation) set
train =h2o.H2OFrame(train)
test = h2o.H2OFrame(test)


In [ ]:
#Modelling for severity
x_Severity=train.columns.remove('Severity')
y_Severity="Severity"
aml_Severity = H2OAutoML(max_models=10, seed=1)

In [ ]:
#Cross validation occurs with this code
#Code also handles conversion of Severity into nominal data (encoding)
aml_Severity.train(x=x_Severity, y=y_Severity, training_frame=train)

In [ ]:
h2o.explain(aml_Severity.leader,train)

In [ ]:
test_pred = test.drop(['Severity'],axis=1)
preds_Severity = aml_Severity.predict(test_pred)

In [ ]:

y_true=test['Severity'].as_data_frame()
y_pred=preds_Severity.as_data_frame()
print("MSE",mean_squared_error(y_true,y_pred['predict'].tolist()))
print("R2_Score",r2_score(y_true,y_pred))


In [ ]:
# Modelling for Distance_mi variable
x_Distance=train.columns.remove('Distance_mi')
y_Distance="Distance_mi"
aml_Distance = H2OAutoML(max_models=10, seed=1)
aml_Distance.train(x=x_Distance, y=y_Distance, training_frame=train)

In [ ]:
h2o.explain(aml_Distance.leader,train)

In [ ]:
test_pred = test.drop(['Distance_mi'],axis=1)
preds_Distance = aml_Distance.predict(test_pred)

In [ ]:
y_true=test['Distance_mi'].as_data_frame()
y_pred=preds_Distance.as_data_frame()
print("MSE",mean_squared_error(y_true,y_pred['predict'].tolist()))
print("R2_Score",r2_score(y_true,y_pred))

In [ ]:

plt.figure(figsize=(10,5))
plt.xlabel('Pressure_in')
plt.ylabel('Distance_mi')
sns.scatterplot(x=df['Pressure_in'],y=df['Distance_mi'])
plt.show()


In [ ]:

plt.figure(figsize=(10,5))
plt.xlabel('Wind_Speed_mph')
plt.ylabel('Distance_mi')
sns.scatterplot(x=df['Wind_Speed_mph'],y=df['Distance_mi'])
plt.show()


In [ ]:
plt.figure(figsize=(10,5))
plt.xlabel('Visibiliy_mi')
plt.ylabel('Distance_mi')
sns.scatterplot(x=df['Visibility_mi'],y=df['Distance_mi'])
plt.show()


In [ ]:
plt.figure(figsize=(10,5))
plt.xlabel('Junction')
plt.ylabel('Distance_mi')
sns.scatterplot(x=df['Junction'],y=df['Distance_mi'])
plt.show()
